In [1]:
#import the libraries

from elasticsearch import Elasticsearch
import json
import re
import pandas as pd
import numpy as np
import bs4
import lxml
from bs4 import BeautifulSoup

In [2]:
import os

In [20]:
os.chdir("text_files_html")

In [21]:
#preparing the data in json format

In [22]:
tag_dict = {}
for file in os.listdir():
    
    if file.endswith("html"):
        f_dict = {}
        key_val = int(str(file)[0:-5])
        f = open(file,"r",encoding="utf-8")
        content = f.read()
        soup = BeautifulSoup(content,"lxml")

        #get title of document

        title = soup.find("div",{"class":"doc_title"})
        if(title):
            title = title.text
        else:
            title = ""


        #get author of document

        author = soup.find("div",{"class":"doc_author"})
        if(author):
            author = author.text
            author = author[8:] #to remove author keyword from the text
        else:
            author = ""

            
        f_dict["title"] = title
        f_dict["author"] = author

        
        
        
        para = {}
        ptext = ""
        #extract the paragraphs from the judgements
        for i in soup.find_all("p"):
            #f_dict[i.get('id')] = i.text
            ptext += i.text
            
        f_dict["content"] = ptext


        tag_dict[key_val] = f_dict
    

In [23]:
os.chdir("..")

In [24]:
with open('data_for_index_p_combined.json','w',encoding="utf-8") as outfile:
    json.dump(tag_dict,outfile,indent=4)

In [12]:
#indexing the data

In [3]:
from elasticsearch import Elasticsearch

In [9]:
#initializing the instance of elasticsearch to localhost at port 9200(default)
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [26]:
#create the index

es.indices.delete(index='judgement_index', ignore=[400, 404])
es.indices.create(index="judgement_index")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'judgement_index'}

In [27]:
#load the json file

file = open("data_for_index_p_combined.json","r",encoding="utf-8")

data = json.load(file)

In [28]:
#insert the data in the index

for i in data.keys():
    es.index(index="judgement_index", doc_type="cases", id=int(i), body=data[i])

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.


In [29]:
def index_func(where_to_search,text_to_search):
    """
    parameters:
    where_to_search: scope of search eg: title, author, etc.
    text_to_search: text that you are looking for in the document
    
    output:
    1. returns, if no output is found
    2. details of judgement, otherwise
    """
    
    #store the result of the query
    result = es.search(index="judgement_index", doc_type="cases", body={"query": {"match": {where_to_search: text_to_search}}})
    
    #extract the number of results from the result object
    num_results = result['hits']['total']['value']
    
    #return if nothing is found
    if num_results == 0:
        print("No result found!")
        return
    else:
        #print the number of results found
        print(str(num_results)+" result/s found!\n\n")
        
        #print the details of the judgement
        for i in range(num_results):
            res = result['hits']['hits'][i]['_source']
            print("RESULT "+str(i+1))
            print("---------")
            print("Document title : "+str(res['title']))
            print("Document author : "+str(res['author']))
            print("Document content\n")
            for i in res.keys():
                if i!='source' and i!='title':
                    print(res[i])
            print("\n\n\n")
            return


text_to_search = input("What do you want to search?: ")
where_to_search = input("Where do you want to search in?(title, author, content): ")
index_func(where_to_search,text_to_search)

What do you want to search?: Astrazeneca
Where do you want to search in?(title, author, content): content
4 result/s found!


RESULT 1
---------
Document title : United Biotech Pvt. Ltd. vs Orchid Chemicals & ... on 18 May, 2012
Document author : A.K.Sikri
Document content

A.K.Sikri
1.    The appellant herein got registration of trademark FORZID
      under No.1144258 dated 18.10.2002 in Class - 5 questioning
      this registration in favour of the appellant, the respondent
      herein filed application for removal of the aforesaid trademark
      FORZID or rectification of register under Sections 9, 11, 18, 57
      and 125 of the Trademarks Act, 1999 (hereinafter referred to
      as „the Act‟). To put it in brief, the removal was sought on the
      ground that the said trademark FORZID was deceptively similar
      to an earlier registered trademark, viz., ORZID (label mark),
      standing registered in the name of the respondent No.1 in
      Class - 5. The plea of the respond

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  del sys.path[0]
